# Multimodale Prompts

In [ ]:
!pip install openai

In [ ]:
# OPENAI KEY lesen
import os
try:
    from google.colab import userdata
    OPENAI_KEY = userdata.get('OPENAI_KEY')
except:
    OPENAI_KEY = os.getenv('OPENAI_KEY')
os.environ['OPENAI_API_KEY'] = OPENAI_KEY


## Bilder verarbeiten

In [ ]:
from openai import OpenAI
client = OpenAI()

Wir senden eine "User"-Message mit dem Bild an die API.

In [ ]:
response = client.chat.completions.create(
  model="gpt-4-vision-preview",
  messages=[
    {
      "role": "user",
      "content": [
        {"type": "text", "text": "Beschreibe das Bild!"},
        {
          "type": "image_url",
          "image_url": {
            "url": "https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg",
          },
        },
      ],
    }
  ],
  max_tokens=300,
)

![image](https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg)

In [ ]:
print(response.choices[0].message.content)

### "Lokale" Bilder verwenden

Neben Bilder-URLs können wir auch lokale Bilder auf der eigenen Festplatte verwenden. Diese müssen "Base64-encoded" an die API gesendet werden.

In [ ]:
from io import BytesIO
from PIL import Image
import base64

In [ ]:
# Bild Base64 encoden + verkleinern (512x512 Pixel)
def encode_image(image_path, size=512):
  img = Image.open(image_path)
  img.thumbnail([size, size],Image.BILINEAR)

  buffered = BytesIO()
  img.save(buffered, format="PNG")
  return base64.b64encode(buffered.getvalue()).decode('utf-8')

In [ ]:
base64_image = encode_image('data/slide.png')
base64_image

In [ ]:
response = client.chat.completions.create(
  model="gpt-4-vision-preview",
  messages=[
    {
      "role": "user",
      "content": [
        {"type": "text", "text": "Beschreibe das Bild!"},
        {
          "type": "image_url",
          "image_url": {
            "url": f"data:image/jpeg;base64,{base64_image}",
          },
        },
      ],
    }
  ],
  max_tokens=300,
)

![image](data/slide.png)

In [ ]:
print(response.choices[0].message.content)

### Qualität

Einfluss auf die Qualität der Ergebnisse hat die Auflösung und die "Detail"-Einstellung der API. Beides hat aber auch Einfluss auf den Preis!

In [ ]:
# 1024x1024 statt 512x512 wie im ersten Aufruf
base64_image = encode_image('data/slide.png', 1024)

In [ ]:
response = client.chat.completions.create(
  model="gpt-4-vision-preview",
  messages=[
    {
      "role": "user",
      "content": [
        {"type": "text", "text": "Extrahiere den Text des Bildes!"},
        {
          "type": "image_url",
          "image_url": {
            "url": f"data:image/jpeg;base64,{base64_image}",
            # Wir setzten die "Detail"-Einstellung auf hoch.
            "detail": "high"
          },
        },
      ],
    }
  ],
  max_tokens=300,
)

In [ ]:
print(response.choices[0].message.content)

### Mehrere Bilder

Es können auch problemlos mehrer Bilder in einer Message geschickt werden. Auch können sich Bilder und Texte abwechseln!

In [ ]:
response = client.chat.completions.create(
  model="gpt-4-vision-preview",
  messages=[
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": "Was ist auf diesen Bildern? Wie unterscheiden sie sich?",
        },
        {
          "type": "image_url",
          "image_url": {
            "url": "https://upload.wikimedia.org/wikipedia/commons/thumb/9/9b/Chin_posing.jpg/702px-Chin_posing.jpg",
          },
        },
        {
          "type": "image_url",
          "image_url": {
            "url": "https://upload.wikimedia.org/wikipedia/commons/thumb/9/9b/Gustav_chocolate.jpg/505px-Gustav_chocolate.jpg",
          },
        },
      ],
    }
  ],
  max_tokens=300,
)

In [ ]:
print(response.choices[0].message.content)